# IBM Data Science Capstone Project

Note: this notebook is dedicated for the IBM Data Science certificate capstone project

### Week 3 - Segmenting and Clustering Neighborhoods in Toronto

In [1]:
from bs4 import BeautifulSoup
import requests
import lxml

import pandas as pd
import numpy as np

Webscrapping - wikipedia

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url)
html = r.text
bs = BeautifulSoup(html)
#print(bs.prettify())  Check out the prettified html

In [3]:
result =[]
for item in bs.find_all('td'):
    result.append(item.get_text())

Data cleaning

In [4]:
#remove unwanted items
result=result[:-33]

In [5]:
postal_code=result[::3]
postal_code = [x[:-1] for x in postal_code]

In [6]:
borough = result[1::3]
borough = [x[:-1] for x in borough]

In [7]:
neighborhood = result[2::3]
neighborhood = [x[:-1] for x in neighborhood]

In [8]:
data = {'PostalCode': postal_code, 'borough': borough, 'neighborhood': neighborhood}
df = pd.DataFrame(data, columns=data.keys())

In [9]:
df.head()

,PostalCode,borough,neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
#ignore rows without an assigned borough
df = df[df.borough!='Not assigned']
df.reset_index(inplace=True, drop=True)
df.head()

,PostalCode,borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
#check if there are any 'Not assigned' neighborhood
sum(df.neighborhood=='Not assigned')

0

In [12]:
df.shape

(103, 3)

Get lat and long coordinates of each neighborhood

In [26]:
coords = pd.read_csv('Geospatial_Coordinates.csv', names=['PostalCode','Latitude','Longitude'], header=0)

In [27]:
coords.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
result = pd.merge(df, coords, on=['PostalCode'])
result.head(11)

,PostalCode,borough,neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
